In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
df = pd.DataFrame([
    {"text": "Apple product with a sleek design.", "category": "Electronics"},
    {"text": "Laptop stand for the kitchen.", "category": "Furniture/Home Decor"},
    {"text": "Chocolate leather boots.", "category": "Footwear/Clothing"},
    {"text": "Wooden cream for surfaces.", "category": "Furniture/Home Decor"},
    {"text": "Natural finish for your lips.", "category": "Beauty/Personal Care"}
])
df

,text,category
0,Apple product with a sleek design.,Electronics
1,Laptop stand for the kitchen.,Furniture/Home Decor
2,Chocolate leather boots.,Footwear/Clothing
3,Wooden cream for surfaces.,Furniture/Home Decor
4,Natural finish for your lips.,Beauty/Personal Care


In [3]:
from ally.skills.labeling.classification import ClassificationSkill


classification_skill = ClassificationSkill(
  name='product_category_classification',
  instruction_template="""
  Label the input text with the following labels:
    "Footwear/Clothing",
    "Electronics",
    "Food/Beverages",
    "Furniture/Home Decor",
    "Beauty/Personal Care"'
  """,
  input_template="{text}"
)

In [4]:
from ally.runtimes.openai import OpenAIRuntime
from app.core.settings import settings


openai_runtime = OpenAIRuntime(
    verbose=True,
    api_key=settings.openai_api_key,
    gpt_model_name="gpt-3.5-turbo",
  )

In [5]:
from ally.agents.base import Agent
from ally.environments.base import BasicEnvironment


agent = Agent(
	skills=classification_skill,
  runtimes={
    'openai': openai_runtime
	},
  teacher_runtimes={
    'openai': openai_runtime
	},
	environment=BasicEnvironment(
			ground_truth_dataset=df,
			ground_truth_columns={'product_category_classification': 'category'}
	)
)

In [6]:
agent.learn()

Applying skill: product_category_classification

['text']

  0%|          | 0/5 [00:00<?, ?it/s]

{'text': 'Apple product with a sleek design.', 'category': 'Electronics'}

 40%|████      | 2/5 [00:04<00:06,  2.13s/it]

...

Comparing predictions to ground truth data ...

 text                       category               product_category_classif…   score    product_category_classi…  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Apple product with a       Electronics            Electronics                 -0.987   True                      
  sleek design.                                                                                                    
  Laptop stand for the       Furniture/Home Decor   Furniture/Home Decor        -4.123   True                      
  kitchen.                                                                                                         
  Chocolate leather boots.   Footwear/Clothing      Footwear/Clothing           -4.123   True                      
  Wooden cream for           Furniture/Home Decor   Furniture/Home Decor        -5.123   True                      
  surfaces.                                                                                                        
  Natural finish for your    Beauty/Personal Care   Beauty/Personal Care        -5.123   True                      
  lips.

No skill to improve found. Stopping learning process.

Train is done!

GroundTruthSignal(match=   product_category_classification
0                             True
1                             True
2                             True
3                             True
4                             True, errors={'product_category_classification': Empty DataFrame
Columns: [predictions, category]
Index: []})